In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install googletrans
!pip install indic_transliteration

     |████████████████████████████████| 61kB 2.7MB/s 
     |████████████████████████████████| 51kB 3.6MB/s 
     |████████████████████████████████| 972kB 6.6MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 102kB 6.7MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=778a2b78134560bdf9a64e14b2bec988b1cbb38e801a92fc0f4a1bf51965deca
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=3e029657ec76d1386537dd20ebc20c772ab0b75f5da27c5eee731a99eb5554fa
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built googletrans contextvars
     |████████████████████████████████| 102kB 3.7MB/s 
     |████████████████████████████████| 665

In [ ]:
import re
import math
import pickle
import random
import pandas as pd
import time
import nltk
import numpy as np
import json
import copy
import sys
import codecs
from collections import *
from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate
from googletrans import Translator

In [ ]:
hospitals = pd.read_csv('./drive/My Drive/new_hospital_1.csv')
hospitals.drop(columns=['Sr_No'],axis=1,inplace=True)
hospitals.fillna(value=0, inplace=True)
translator = Translator()
col = list(hospitals.columns)
print(col)
print(len(col))

['Location_Coordinates', 'Location', 'Hospital_Name', 'Hospital_Category', 'Hospital_Care_Type', 'Discipline_Systems_of_Medicine', 'Address_Original_First_Line', 'State', 'District', 'Subdistrict', 'Pincode', 'Telephone', 'Mobile_Number', 'Emergency_Num', 'Ambulance_Phone_No', 'Bloodbank_Phone_No', 'Foreign_pcare', 'Tollfree', 'Helpline', 'Hospital_Fax', 'Hospital_Primary_Email_Id', 'Hospital_Secondary_Email_Id', 'Website', 'Specialties', 'Facilities', 'Accreditation', 'Hospital_Regis_Number', 'Registeration_Number_Scan', 'Nodal_Person_Info', 'Nodal_Person_Tele', 'Nodal_Person_Email_Id', 'Town', 'Subtown', 'Village', 'Establised_Year', 'Ayush', 'Miscellaneous_Facilities', 'Number_Doctor', 'Num_Mediconsultant_or_Expert', 'Total_Num_Beds', 'Number_Private_Wards', 'Num_Bed_for_Eco_Weaker_Sec', 'Empanelment_or_Collaboration_with', 'Emergency_Services', 'Tariff_Range', 'State_ID', 'District_ID']
47


In [ ]:
translator = Translator()
data = ['Tamil Nadu', 'Maharashtra', 'Trivandrum', 'Salem', 'Trichy', 'Coimbatore', 'Pondicherry', '(0933)401239']
for place in data:
    text = translator.translate(place, src='en', dest='hi')
    print(text.text, end='  ')
    print(transliterate(place, sanscript.ITRANS, sanscript.DEVANAGARI))

# So we need to process the text before conversion as it is difficult to convert directly
print(translator.translate('104,2ND AGRAHARAM,NEAR OLD BUS STAND', src='en', dest='hi').text)

तमिलनाडु  टमिल् णदु
महाराष्ट्र  ंअहरश्त्र
त्रिवेंद्रम  ट्रिवन्द्रुम्
सलेम  Sअलेम्
त्रिची  ट्रिच्य्
कोयंबटूर  Cओइम्बतोरे
पांडिचेरी  Pओन्दिचेर्र्य्
(0933) 401239  (०९३३)४०१२३९
104,2ND AGRAHARAM, NEAR OLD BUS STAND


In [ ]:
hospitals[:][:6]

,Location_Coordinates,Location,Hospital_Name,Hospital_Category,Hospital_Care_Type,Discipline_Systems_of_Medicine,Address_Original_First_Line,State,District,Subdistrict,Pincode,Telephone,Mobile_Number,Emergency_Num,Ambulance_Phone_No,Bloodbank_Phone_No,Foreign_pcare,Tollfree,Helpline,Hospital_Fax,Hospital_Primary_Email_Id,Hospital_Secondary_Email_Id,Website,Specialties,Facilities,Accreditation,Hospital_Regis_Number,Registeration_Number_Scan,Nodal_Person_Info,Nodal_Person_Tele,Nodal_Person_Email_Id,Town,Subtown,Village,Establised_Year,Ayush,Miscellaneous_Facilities,Number_Doctor,Num_Mediconsultant_or_Expert,Total_Num_Beds,Number_Private_Wards,Num_Bed_for_Eco_Weaker_Sec,Empanelment_or_Collaboration_with,Emergency_Services,Tariff_Range,State_ID,District_ID
0,"11.6233774, 92.7264828","GB Pant Road, City Centre, Port Blair","G B Pant Hospital, Port Blair",Public/ Government,Hospital,Allopathic,"GB Pant Road, City Centre, Port Blair",Andaman and Nicobar Islands,South Andaman,0,744103,"03192 233665, 03192 246058, 03192 233455, 0319...",0,0,0,0,0,0,0,0,0,0,http://www.and.nic.in/C_charter/health/dhs/ind...,"Surgery, Medicine, Gynaecology, Pediatrics, EN...",0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,35,640
1,"17.4274003, 78.4311174","Road No.1, Banjara Hills","Care Hospital, Banjara",Private,Hospital,Allopathic,"Road No.1, Banjara Hills",Andhra Pradesh,Hyderabad,0,500034,"040 30418888, 040 66668888, 040 23234444",0,040 30418888,0,0,0,18001086666,0,0,info@carehospitals.com,0,www.carehospitals.com,"Anaesthesiology, Bio-Chemistry, Cardiac Yoga,\...","Ambulance, Blood Bank,\nCasualty, Diagnostic, ...",0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,28,0
2,"17.4230892, 78.42647","Kallam Anji Reddy Campus, L V Prasad Marg, Ban...",L V Prasad Eye Institute,Private,Medical College / Institute,Allopathic,"Kallam Anji Reddy Campus, L V Prasad Marg, Ban...",Andhra Pradesh,Hyderabad,0,500034,"040 30612345, 040 39892020",0,"040 30612100, 09347573377",0,0,0,0,0,0,appointment@lvpei.org,0,www.lvpei.org,Ophthalmology,0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,28,0
3,"17.4325235, 78.4070152",Jubilee Hills,Apollo Hospitals,Private,Hospital,Allopathic,Jubilee Hills,Andhra Pradesh,Hyderabad,0,500033,040 23607777,0,0,0,0,0091 4043441066,18605001066,0,0,apollohealthcity@apollohospitals.com,0,www.apollohospitals.com,"Cardiology,\n Cardiothoracic Surgery,\n Cosmet...","Inpatient, Accommodation, Food and Beverage Se...",0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,28,0
4,"17.406396, 78.471592","5 - 9 - 22, Secretariat Road",Mediciti Hospital,Private,Hospital,Allopathic,"5 - 9 - 22, Secretariat Road",Andhra Pradesh,Hyderabad,0,500063,040 23231111,0,0,0,0,0,0,0,0,info@medicitihospitals.com,0,www.medicitihospitals.com,"Anaesthesiology,\n Cardiology,\n Cardiothoraci...","Accident and Emergency Ward,\nCardiothoracic, ...",0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,28,0
5,"17.430016, 78.4897935","Secunderabad, 1 - 8 - 31/1, Minister Road, Sec...","KIMS Hospital, Secunderabad",Private,Hospital,Allopathic,"Secunderabad, 1 - 8 - 31/1, Minister Road, Sec...",Andhra Pradesh,Hyderabad,0,500003,"040 44885000, 040 44885184",0,9849990401,0,0,0,0,0,0,info@kims.co.in,0,www.kimshospitals.com,"Accident and Emergency, Anaesthesiology, Baria...","24 hours Ambulance service,\nRound the clock p...",0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,28,0


# Dictionary Creation

In [ ]:
keyword_list = {}
word_list = defaultdict(list)

mapping_file_name='hindi_template.txt'
words_map=open(mapping_file_name)
words=words_map.read()
words=words.split("\n")

key=""
value=[]
for word in words:
    if word == "" or word == " ":
        continue
    if "=" in word:
        if key != "":
            word_list[key]=value
        word = word.split("=")
        key = word[0]
        value = []
    else:
        value.append(word)
word_list[key] = value
for key in word_list.keys():
    if key != 'Hospital_Name':
        print(word_list[key])

['इसका पिन कोड है : $PINCODE । ']
['यह अस्पताल $District जिले के अंतर्गत आता है। ', 'यह अस्पताल $District जिले में आने वाले प्रसिद्ध अस्पतालों में से एक है। ', '$District जिले में आने वाले कई नामी अस्पतालों में इसकी गिनती की जाती है। ']
['कृपया नीचे उल्लिखित अस्पताल का पता खोजें $Location । ', 'नीचे उद्धृत अस्पताल का पता है $Location । ', 'किसी भी परामर्श या आपातकाल के मामले में, कृपया यहाँ अस्पताल जाएँ: $Location । ']
['यह अस्पताल राज्य $State में स्थित है| ']
['इसका पिन कोड है : $Pincode | ']
['यह एक $Hospital_Category का अस्पताल है। ', 'यह $Hospital_Category में आने वाले कई नामी अस्पतालों में से एक है। ', 'यह अस्पताल $Hospital_Category के लिए मशहूर है। ']
['यह $Hospital_Care_Type एक बहुत ही प्रमुख और प्रसिद्ध माना जाता है। ', 'यह $Hospital_Care_Type जनता के बीच सबसे पसंदीदा है। ', 'यह शहर में सबसे अधिक अनुशंसित $Hospital_Care_Type है। ']
['अपने क्षेत्र में सर्वश्रेष्ठ होने के नाते, यह अस्पताल $Discipline_Systems_of_Medicine दवा का अभ्यास करता है। लेकिन इस', 'अस्पताल में दवा के विभिन

# Infobox keys

In [ ]:
INFO = []
with open('./infobox_raw.txt', 'r') as file:
    lines = file.readlines()
    for l in lines:
        sp = l.split()
        if len(sp) >= 3 and sp[2] == '=':
            INFO.append(sp[1])

with open('./infobox.txt', 'w') as file:
    st = "\n".join(INFO)
    file.write(st)
print(INFO)

['name', 'org/group', 'logo', 'logo_size', 'logo_alt', 'image', 'image_size', 'alt', 'caption', 'coordinates', 'location', 'region', 'state', 'country', 'healthcare', 'funding', 'type', 'religious_affiliation', 'affiliation', 'patron', 'network', 'standards', 'emergency', 'beds', 'speciality', 'helipad', 'h1-number', 'h1-length-f', 'h1-length-m', 'h1-surface', 'publictransit', 'former-names', 'constructed', 'opened', 'closed', 'demolished', 'website', 'other_links', 'module']


In [ ]:
def infobox(current_row, lang):
    result = []
    result.append("{{short description|Hospital in India}}")
    result.append("{{Use Indian English|date=November 2020}}")
    result.append("{{Infobox Hospital")
    infobox_dict = {}
    with open("./infobox_map.txt", "r") as file:
        lines = file.readlines()
        for l in lines:
            sp = l.split()
            if len(sp) >= 3:
                infobox_dict[sp[0]] = sp[2]
    
    with open("./infobox.txt", "r") as file:
        lines = file.readlines()
        good = True
        for l in lines:
            st = l.strip()
            temp = st
            st = "| " + st.capitalize()
            done = False
            if temp in infobox_dict.keys():
                done = True
                if temp == "coordinates":
                    arr = current_row[infobox_dict[temp]].split(',')
                    if len(arr) == 2:
                        st += " = {{coord|"+str(arr[0])+"|" + str(arr[1])+"|region:US-MD|format=dms|display=inline}}"
                    else:
                        good = False
                elif temp == "website":
                    st += " = {{url|"+str(current_row[infobox_dict[temp]])+"}}" 
                else:
                    st += " = " + str(current_row[infobox_dict[temp]])
            if not done:
                st += " = "
            if temp == "country":
                st += "भारत"
            if good:
                result.append(st)
    result.append("}}")
    st = "\n".join(result)
    return st

# Generate Articles

In [ ]:
should_translate = dict()
should_transliterate = dict()

with open("./translate.txt", "r") as file:
    lines = file.readlines()
    for l in lines:
        arr = l.split()
        if arr[2] == "True":
            should_translate[arr[0]] = True
        else:
            should_translate[arr[0]] = False

with open("./transliterate.txt", "r") as file:
    lines = file.readlines()
    for l in lines:
        arr = l.split()
        if arr[2] == "True":
            should_transliterate[arr[0]] = True
        else:
            should_transliterate[arr[0]] = False
def completed_sentence(sentence, substitute):
    ''' Replace $ with the given word '''
    words = sentence.split()
    new_words = []
    for word in words:
        if word[0] == '$':
            new_words.append(str(substitute))
        else:
            new_words.append(str(word))
    return ' '.join(new_words)

def create_article(current_row, lang):
    ''' current_row is a dictionary with keys from the data frame '''
    article = ""
    for key in current_row.keys():
        if should_translate[key]:
            # pass
            translated = translator.translate(str(current_row[key]), src='en', dest=lang)
            current_row[key] = translated.text
        if should_transliterate[key]:
            translated = transliterate(str(current_row[key]), sanscript.ITRANS, sanscript.DEVANAGARI)
            current_row[key] = translated
        ind = int(random.randint(0, len(word_list[key]) - 1))
        article += completed_sentence(word_list[key][ind], current_row[key])
    
    info = infobox(current_row, lang)
    info += "\n"
    info += article
    return info

In [ ]:
to_include = []
with open("./Key_list.txt", "r") as file:
    lines = file.readlines()
    for l in lines:
        st = l.strip()
        to_include.append(st)
print(to_include)
print(len(to_include))

['Location', 'Hospital_Name', 'Hospital_Category', 'Hospital_Care_Type', 'Discipline_Systems_of_Medicine', 'State', 'District', 'Pincode', 'Telephone', 'Hospital_Primary_Email_Id', 'Website', 'Specialties', 'Facilities', 'Location_Coordinates']
14


# Generate Till infobox

In [ ]:
position = dict()
position['1st'] = 'first'
position['2nd'] = 'second'
position['3rd'] = 'third'
position['4th'] = 'fourth'
position['6th'] = 'sixth'
position['7th'] = 'seventh'
position['8th'] = 'eighth'
position['9th'] = 'ninth'

def process(data):
    '''process for better translation'''
    data = data.split()
    new_data = []
    for d in data:
        s = ""
        d = list(d)
        dont = False
        for ind in range(len(d)):
            if d[ind] == '\\':
                if ind+1 < len(d) and d[ind+1] == 'n':
                    dont = True
                else:
                    s += str(d[ind])
            else:
                if d[ind] == 'n' and dont:
                    dont = False 
                else:
                    s += str(d[ind])
        new_data.append(s)
    final = []
    for d in new_data:
        d = str(d)
        d = d.lower()
        if d in position:
            final.append(position[d])
        else:
            final.append(d)
    return ' '.join(final)
  

for i in range(4):
    ''' Generate Articles'''
    current_row = dict()
    bad = False
    for k in col:
        if k in to_include:
            val = process(str(hospitals[k][i]))
            current_row[k] = val
            if val == "0" or val == "0.0":
                bad = True
    if not bad:
        print("{{-start-}}")
        print('')
        article = create_article(current_row, lang='hi')
        print(article)
        print("{{-stop-}}")
        print('----------------------')


{{-start-}}

{{short description|Hospital in India}}
{{Use Indian English|date=November 2020}}
{{Infobox Hospital
| Name = केयर अस्पताल, बंजारा
| Org/group = 
| Logo = 
| Logo_size = 
| Logo_alt = 
| Image = 
| Image_size = 
| Alt = 
| Caption = 
| Coordinates = {{coord|17.4274003| 78.4311174|region:US-MD|format=dms|display=inline}}
| Location = रोअद् नो।१, बन्जर हिल्ल्स्
| Region = 
| State = आंध्र प्रदेश
| Country = भारत
| Healthcare = 
| Funding = 
| Type = अस्पताल
| Religious_affiliation = 
| Affiliation = 
| Patron = 
| Network = 
| Standards = 
| Emergency = 
| Beds = 
| Speciality = 
| Helipad = 
| H1-number = 
| H1-length-f = 
| H1-length-m = 
| H1-surface = 
| Publictransit = 
| Former-names = 
| Constructed = 
| Opened = 
| Closed = 
| Demolished = 
| Website = {{url|www.carehospitals.com}}
| Other_links = 
| Module = 
}}
इसके भौगोलिक निर्देशांक ह : 17.4274003, 78.4311174 ।कृपया नीचे उल्लिखित अस्पताल का पता खोजें रोअद् नो।१, बन्जर हिल्ल्स् ।केयर अस्पताल, बंजारा अस्पताल एक भार

In [ ]:
print("According to which field would you like to search")
column = input()
print("Please specify the value for this field")
val = input()
count = 0
for index, row in hospitals.loc[hospitals[column] == val].iterrows():
    current_row = dict()
    bad = False
    for k in col:
        if k in to_include:
            val = process(str(row[k]))
            current_row[k] = val
            if val == "0" or val == "0.0":
                bad = True
                
    if not bad:
        print("START")
        article = create_article(current_row, lang='hi')
        print(article)
        print("STOP")
        print('----------------------')
        count += 1
        if count == 5:
            break
    

According to which field would you like to search
District
Please specify the value for this field
Hyderabad
START


AttributeError: ignored

In [ ]:
arr = "8, 9".split(',')
print(arr)

['8', ' 9']
